## 逻辑回归模型

注意，区分是否为线性模型，主要是看一个乘法式中自变量前的系数w，若w只影响一个自变量（或决策边界为线性），则为线性模型，而一旦有自变量被超过一个参数影响，则为非线性

**机器学习模型训练的步骤**:

数据模块（数据采集，清洗，处理等）

建立模型（各种模型的建立）

损失函数的选择（根据不同的任务选择不同的损失函数），有了loss就可以求取梯度

得到梯度之后，我们会选择某种优化方式去进行优化

然后迭代训练

In [8]:
import torch
"""数据生成"""
torch.manual_seed(1)

sample_nums = 100
mean_value = 1.7
bias = 1

n_data = torch.ones(sample_nums, 2) # 100行，2列


In [9]:
x0 = torch.normal(mean_value*n_data, 1) + bias  # 类别0  数据shape=(100,2)
y0 = torch.zeros(sample_nums)   # 类别0， 数据shape=(100, 1)
x1 = torch.normal(-mean_value*n_data, 1) + bias   # 类别1， 数据shape=(100,2)
y1 = torch.ones(sample_nums)    # 类别1  shape=(100, 1)

train_x = torch.cat([x0, x1], 0)
train_y = torch.cat([y0, y1], 0)

In [10]:
"""建立模型"""
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.features = torch.nn.Linear(2, 1)  # Linear 是module的子类，是参数化module的一种，与其名称一样，表示着一种线性变换。输入2个节点，输出1个节点
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.features(x) # linear feature
        x = self.sigmoid(x) # sigmoid function
        
        return x

lr_net = LR()     # 实例化逻辑回归模型

In [11]:
"""选择损失函数"""
loss_fn = torch.nn.BCELoss()


In [12]:
"""选择优化器"""
lr = 0.01
optimizer = torch.optim.SGD(lr_net.parameters(), lr=lr, momentum=0.9)


In [ ]:
"""模型训练"""
from matplotlib import pyplot as plt
import numpy as np


for iteration in range(1000):
    
    # 前向传播
    y_pred = lr_net(train_x)
    
    # 计算loss
    loss = loss_fn(y_pred.squeeze(), train_y)
    
    # 反向传播
    loss.backward()
    
    # 更新参数
    optimizer.step()
    
    # 清空梯度
    optimizer.zero_grad()
    
    # 绘图
    if iteration % 20 == 0:

        mask = y_pred.ge(0.5).float().squeeze()  # 以0.5为阈值进行分类
        correct = (mask == train_y).sum()  # 计算正确预测的样本个数
        acc = correct.item() / train_y.size(0)  # 计算分类准确率

        plt.scatter(x0.data.numpy()[:, 0], x0.data.numpy()[:, 1], c='r', label='class 0')
        plt.scatter(x1.data.numpy()[:, 0], x1.data.numpy()[:, 1], c='b', label='class 1')

        w0, w1 = lr_net.features.weight[0]
        w0, w1 = float(w0.item()), float(w1.item())
        plot_b = float(lr_net.features.bias[0].item())
        plot_x = np.arange(-6, 6, 0.1)
        plot_y = (-w0 * plot_x - plot_b) / w1

        plt.xlim(-5, 7)
        plt.ylim(-7, 7)
        plt.plot(plot_x, plot_y)

        plt.text(-5, 5, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 20, 'color': 'red'})
        plt.title("Iteration: {}\nw0:{:.2f} w1:{:.2f} b: {:.2f} accuracy:{:.2%}".format(iteration, w0, w1, plot_b, acc))
        plt.legend()

        plt.show()
        plt.pause(0.5)

        if acc > 0.99:
            break


首先基于前面的张量的知识我们又更进一步，学习了计算图的机制，计算图说白了就是描述运算过程的图， 有了这个图梯度求导的时候非常方便。 然后学习了Pytorch的动态图机制，区分了一下动态图和静态图。 然后学习了Pytorch的自动求导机制，认识了两个比较常用的函数torch.autograd.backward()和torch.autograd.grad()函数， 关于自动求导要记得三个注意事项： 梯度手动清零，叶子节点不能原位操作，依赖于叶子节点的节点默认是求梯度。 最后我们根据上面的所学知识建立了一个逻辑回归模型实现了一个二分类的任务
